In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_ablation_nystrom as file

In [2]:
# Args setup
ntrial = 50
n = 100
ntest = 100
d = 5
noise = 0.5
features = 1000
samples = 1000
calibrate = [False, True, True]
sample_split = [False, False, True]
marginal_loss = False
retrain_hypers = False
kernel = "gaussian"

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
    for i in range(len(calibrate)):
            f = client.submit(file.main,seed,n,ntest,d,noise,
                              calibrate = calibrate[i],
                              sample_split = sample_split[i],
                              marginal_loss = marginal_loss,
                              retrain_hypers = retrain_hypers,
                              features = features, samples = samples,
                              kernel = "gaussian")
            futures += [f]

In [5]:
# Check on futures
futures

[<Future: pending, key: main-27bd7125c282ffbd91e1dfa625424aed>,
 <Future: pending, key: main-c971810708cdcccf7d8611950c6d1d37>,
 <Future: pending, key: main-7047d156651f438a77febfbac298510c>,
 <Future: pending, key: main-daf616d49d720b3ce585a12adc37485b>,
 <Future: pending, key: main-9a5fe3f6a36e876f64f0b6478ce03f01>,
 <Future: pending, key: main-c8992aed10813933bb294145bdd6aa94>,
 <Future: pending, key: main-06fd61e94d006e190a6f2646f0f08516>,
 <Future: pending, key: main-5b014c0fd28f438d5fc27a30e58c5c0b>,
 <Future: pending, key: main-da9d5746c8ea892c8fcd001855b0c9fd>,
 <Future: pending, key: main-fc0cf4add49b6b41ac65182c8470d675>,
 <Future: pending, key: main-306fb28834859308aa715f59fac15c68>,
 <Future: pending, key: main-ea806b5bf8ee7f9bbd37c2d387fe0297>,
 <Future: pending, key: main-4ef1316523d024cb0535b98bea47a580>,
 <Future: pending, key: main-3bcb71016bf4e19b35d50b1c456c2e34>,
 <Future: pending, key: main-06624b1a9b36dcadd46712ed5eab7712>,
 <Future: pending, key: main-611c42f78e6

In [6]:
# Getting results
results = client.gather(futures)

In [7]:
# Closing client
client.close()
cluster.close()

In [8]:
# Saving results
torch.save(obj = results,
           f = "ablation_nystrom_ntrial={0}_n={1}_d={2}_noise={3}.pt".format(ntrial,n,d,noise))